In [33]:
import pymysql


In [35]:
import pandas as pd
address_code = pd.read_excel('E:/workplace/py36/采集区域经纬度判断/repeat100m_combined.xlsx')

In [46]:
cursor.execute("select address_code_number,address_name,collected_latitude,collected_longitude,collector_id,manager_id,create_time,update_time from store.population_collector_address where address_code_number in "+str(tuple(address_code['address_code_number'])))
result =  cursor.fetchall()
village_with_latlon = pd.DataFrame(data = result)

In [47]:
village_with_latlon

,address_code_number,address_name,collected_latitude,collected_longitude,collector_id,manager_id,create_time,update_time
0,430611103005,君山区钱粮湖镇托龙山社区,29.446405436197917,112.67197374131945,4306119,4306235,2020-09-08 10:29:45,2020-09-11 12:39:32
1,430611103201,君山区钱粮湖镇西北湖村,29.447857801649306,112.6728480360243,4306119,4306235,2020-09-08 10:29:45,2020-09-11 12:39:41
2,430611104002,君山区良心堡镇福华社区,29.355077039930556,112.74698459201389,43061120,4306235,2020-09-08 10:29:45,2020-09-11 19:01:59
3,430611104201,君山区良心堡镇维新村,29.330127495659724,112.73299235026042,43061120,4306235,2020-09-08 10:29:45,2020-09-11 19:02:06
4,430611104202,君山区良心堡镇福星村,29.34498779296875,112.74654052734375,43061120,4306235,2020-09-08 10:29:45,2020-09-11 19:02:14
...,...,...,...,...,...,...,...,...
75,430903402014,赫山区龙岭工业园八一社区,28.343443467881944,112.51765299479166,43090350,4309035,2020-09-08 10:29:45,2020-09-09 07:08:01
76,430903002008,赫山区桃花仑街道大桃路社区,28.570528030395508,112.36111450195312,43090351,4309032,2020-09-08 10:29:45,2020-09-12 14:21:46
77,430903002005,赫山区桃花仑街道康复路社区,28.570571899414062,112.36109924316406,43090351,4309032,2020-09-08 10:29:45,2020-09-12 14:21:54
78,430903111261,赫山区兰溪镇双港子村,28.54585747612847,112.45939507378472,43090364,4309037,2020-09-08 10:29:45,2020-09-12 09:56:30


In [48]:
from geopy import distance
def get_sl_distance(lon1,lat1,lon2,lat2):
    return distance.distance([lat1, lon1], [lat2, lon2]).km

In [59]:
get_sl_distance(112.361114501953,28.5705280303955,112.351388,28.574204)

1.03515542537033

In [49]:
def find_nearest_village(v_code,v_lat,v_lon,county):
    temp = county[county['address_code_number'] != v_code].copy()
    temp['sl_distance'] = temp.apply(lambda df: get_sl_distance(v_lon,v_lat,df['collected_longitude'],df['collected_latitude']),axis=1)
    target = temp[temp['sl_distance'] < 0.1]
    target['address_code_number'] = target['address_code_number'].astype(str)
    out = target['address_code_number'].values.tolist()
    if len(out) <1:
        return 1
    else:
        return ','.join(out)

In [50]:
village_with_latlon['repetition'] = village_with_latlon.apply(lambda df:find_nearest_village(df['address_code_number'],df['collected_latitude'],df['collected_longitude'],village_with_latlon),axis=1)

In [51]:
dt1 = village_with_latlon.loc[village_with_latlon['repetition'] != 1]

In [54]:
dt1

,address_code_number,address_name,collected_latitude,collected_longitude,collector_id,manager_id,create_time,update_time,repetition
70,430903104240,赫山区欧江岔镇欧江岔村,28.488096245659722,112.62327555338541,43090334,4309036,2020-09-08 10:29:45,2020-09-09 15:32:19,430903104003
73,430903104003,赫山区欧江岔镇罗湖社区,28.488096245659722,112.62327555338541,43090334,4309036,2020-09-08 10:29:45,2020-09-09 15:33:04,430903104240
74,430903402016,赫山区龙岭工业园南岳坪社区,28.343339572482638,112.51732096354166,43090348,4309035,2020-09-08 10:29:45,2020-09-09 10:35:14,430903402014
75,430903402014,赫山区龙岭工业园八一社区,28.343443467881944,112.51765299479166,43090350,4309035,2020-09-08 10:29:45,2020-09-09 07:08:01,430903402016
76,430903002008,赫山区桃花仑街道大桃路社区,28.570528030395508,112.36111450195312,43090351,4309032,2020-09-08 10:29:45,2020-09-12 14:21:46,430903002005
77,430903002005,赫山区桃花仑街道康复路社区,28.570571899414062,112.36109924316406,43090351,4309032,2020-09-08 10:29:45,2020-09-12 14:21:54,430903002008


In [53]:
dt1.to_excel('./采集区域经纬度判断/重新采集repeat100m.xlsx')

In [10]:
from impala.dbapi import connect
conn = connect(host='192.168.1.227 ', port=10000, database='dw_stat',user='liyufan',password='Juno#2021',auth_mechanism='PLAIN')
cursor = conn.cursor()

ModuleNotFoundError: No module named 'impala.dbapi'

In [ ]:
from pyhive import hive 
conn = hive.Connection(host='172.21.195.18', port=10000, database='dw_stat',user='liyufan',password='Juno#2021',auth_mechanism='PLAIN')
cursor = conn.cursor()

In [ ]:
cursor.execute("select store_no,count(*),sum(total_qty) from daily_store_order_info where dt>'20200701' and store_no in "+str(tuple(storecode_list))+' group by store_no')
result =  cursor.fetchall()